In [9]:
import pickle

#load pickle file
with open('/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/MIL_3D_GNN/tunning_gnn.pkl', 'rb') as f:
    study = pickle.load(f)

In [10]:
#Load configuration of the best model
const_config = {"node_dim_2d": 34,
                "node_dim_3d": 18,
                "edge_dim_2d": 11,
                "edge_dim_3d": 9,
                "mlp_layers_instance":1,
                "n_block": 3,
                "pooling_every_n": 1}
tune_dict = study.trials[13].params
config = {**const_config, **tune_dict}

In [11]:
from MIL_3D_GNN import instance_GNN
import torch
model = instance_GNN(config)
path = '/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/MIL_3D_GNN/mil_3d_gnn.pth'
checkpoint = torch.load(path, map_location="cpu")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [13]:
import sys
sys.path.append("..")
from conformation_encode.GraphDataset import BagMoleculeDataset
data_path ="/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/dataset/Featurized_data/MIL_3D_GNN/train_test_val/"
train_dataset = BagMoleculeDataset(root = data_path, filename = "cdr1_train.csv")
test_dataset = BagMoleculeDataset(root = data_path, filename = "cdr1_test.csv", test = True)
valid_dataset = BagMoleculeDataset(root = data_path, filename = "cdr1_valid.csv", valid = True)
# #Load the hard test set
data_path ="/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/dataset/Featurized_data/MIL_3D_GNN/hard_test/"
hard_test_dataset = BagMoleculeDataset(root = data_path, filename = "cdr1_hard_test.csv", test = True)

*****


In [14]:
from torch_geometric.data import DataLoader
from MIL_3D_GNN import instance_GNN
import torch
seed = 42
torch.manual_seed(seed)
train_loader = DataLoader(train_dataset, batch_size = 1, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False)
valid_loader = DataLoader(valid_dataset, batch_size = 1, shuffle = False)
hard_test_loader = DataLoader(hard_test_dataset, batch_size=1, shuffle=False)

In [15]:
import random
import numpy as np
from tqdm import tqdm
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def eval_train(loader):
    seed_everything(seed)
    loss_func = torch.nn.BCELoss
    model = instance_GNN(config)
    path = '/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/MIL_3D_GNN/mil_3d_gnn.pth'
    checkpoint = torch.load(path, map_location="cpu")
    model.load_state_dict(checkpoint['model_state_dict'])
    model.train()
    all_labels = []
    all_probas = []
    all_preds = []
    for data in tqdm(loader):
        data = data.to(device)
        #y_true = data.y
        with torch.no_grad():
            output = model(data)
            all_probas.append(output.cpu().detach().numpy())
            all_preds.append(np.rint(output.cpu().detach().numpy()))
            all_labels.append(data.y.cpu().detach().numpy())
    all_probas = np.concatenate(all_probas).ravel()
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_labels, all_preds, all_probas
def eval(loader):
    seed_everything(seed)
    model = instance_GNN(config)
    path = '/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/MIL_3D_GNN/mil_3d_gnn.pth'
    checkpoint = torch.load(path, map_location="cpu")
    model.load_state_dict(checkpoint['model_state_dict'])
    loss_func = torch.nn.BCELoss
    model.eval()
    validation_loss = 0
    all_labels = []
    all_probas = []
    all_preds = []
    for data in tqdm(loader):
        data = data.to(device)
        y_true = data.y
        #weights = self.generate_weights(y_true).to(self.device)
        with torch.no_grad():
            output = model(data)
            loss = loss_func()(output, data.y.float())
            validation_loss += loss.item()
            all_probas.append(output.cpu().detach().numpy())
            all_preds.append(np.rint(output.cpu().detach().numpy()))
            all_labels.append(data.y.cpu().detach().numpy())
        
    all_probas = np.concatenate(all_probas).ravel()
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_labels, all_preds, all_probas

In [16]:
seed_everything(seed)
y_train, y_pred_train, y_probas_train = eval_train(train_loader)
y_hard_test, y_hard_pred, y__hard_probas = eval(hard_test_loader)
y_test, y_pred, y_probas = eval(test_loader)
y_valid, y_pred_valid, y_probas_valid = eval(valid_loader)

100%|██████████| 106/106 [00:24<00:00,  4.35it/s]


In [19]:
y_probas_no_sampling = y_probas_train[:423]
np.savetxt("gnn_probas_no_sampling.txt", y_probas_no_sampling)
np.savetxt("gnn_probas_test.txt", y_probas)
np.savetxt("gnn_probas_valid.txt", y_probas_valid)
np.savetxt("gnn_probas_hard_test.txt", y__hard_probas)

In [2]:
import optuna
optuna.visualization.plot_param_importances(study)

In [3]:
optuna.visualization.plot_optimization_history(study)

In [5]:
optuna.visualization.plot_parallel_coordinate(study)